<a href="https://colab.research.google.com/github/Zizhenz/stat507-coursework/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print("GPU is available:", tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU is available: True


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
#model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")
#pre_model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased",num_labels=4)
pre_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch

In [4]:
df_train=pd.read_csv('twitter_training.csv')
df_test=pd.read_csv('twitter_validation.csv')
df_train.columns=['id','entity','sentiment','content']
df_test.columns=['id','entity','sentiment','content']
df_train.head()

FileNotFoundError: [Errno 2] No such file or directory: 'twitter_training.csv'

In [ ]:
df_train.dropna(inplace=True)
df_train.drop_duplicates(inplace=True,subset=['content'])

In [ ]:
import re
from transformers import DataCollatorWithPadding
def clean_text(text):
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text
df_train['clean_text']=df_train['content'].apply(lambda x:clean_text(x))
df_test['clean_text'] = df_test['content'].apply(clean_text)
df_train.head()
train_texts=df_train['clean_text'].tolist()
val_texts=df_test['clean_text'].tolist()
sentiment_mapping = {
    "Irrelevant": 0,
    "Negative": 1,
    "Neutral": 2,
    "Positive": 3
}

train_labels = list(map(sentiment_mapping.get, df_train['sentiment'].tolist()))
val_labels = list(map(sentiment_mapping.get, df_test['sentiment'].tolist()))

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")

In [ ]:
train_dataset = torch.utils.data.TensorDataset(
    train_encodings["input_ids"], train_encodings["attention_mask"], torch.tensor(train_labels)
)
val_dataset = torch.utils.data.TensorDataset(
    val_encodings["input_ids"], val_encodings["attention_mask"], torch.tensor(val_labels)
)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=16)

In [ ]:
from sklearn.metrics import classification_report
def evaluate_model(model, dataloader):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [b.to("cuda") for b in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, dim=-1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    predictions = np.array(predictions).flatten()
    true_labels = np.array(true_labels).flatten()
    print("Classification Report:")
    print(classification_report(true_labels, predictions, digits=4))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pre_model.to(device)

evaluate_model(pre_model, val_dataloader)

Classification Report:
              precision    recall  f1-score   support

           0     0.1627    0.4854    0.2438       171
           1     0.2515    0.4624    0.3258       266
           2     0.0000    0.0000    0.0000       285
           3     0.0000    0.0000    0.0000       277

    accuracy                         0.2062       999
   macro avg     0.1036    0.2369    0.1424       999
weighted avg     0.0948    0.2062    0.1285       999



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
if isinstance(train_dataset, torch.utils.data.TensorDataset):
    train_dataset = [
        {"input_ids": input_ids, "attention_mask": attention_mask, "labels": label}
        for input_ids, attention_mask, label in train_dataset
    ]
    val_dataset = [
        {"input_ids": input_ids, "attention_mask": attention_mask, "labels": label}
        for input_ids, attention_mask, label in val_dataset
    ]

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    save_strategy="steps",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=2000,
    save_total_limit=3,
    logging_dir='./logs',
    logging_steps=500,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss
500,1.133300,0.969740
1000,0.948900,0.756809
1500,0.832800,0.599800
2000,0.773000,0.507129
2500,0.676200,0.476298
3000,0.623600,0.317414
3500,0.530600,0.250049
4000,0.485100,0.197963
4500,0.384600,0.176160
5000,0.249900,0.153932


TrainOutput(global_step=13032, training_loss=0.3451254393624118, metrics={'train_runtime': 1240.8191, 'train_samples_per_second': 168.01, 'train_steps_per_second': 10.503, 'total_flos': 1.371293666915328e+16, 'train_loss': 0.3451254393624118, 'epoch': 3.0})

In [ ]:
trainer.save_model("./saved_model")
tokenizer.save_pretrained("./saved_model")

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [ ]:
evaluate_model(model, val_dataloader)

Classification Report:
              precision    recall  f1-score   support

           0     0.9817    0.9415    0.9612       171
           1     0.9769    0.9549    0.9658       266
           2     0.9517    0.9684    0.9600       285
           3     0.9474    0.9747    0.9609       277

    accuracy                         0.9620       999
   macro avg     0.9644    0.9599    0.9620       999
weighted avg     0.9624    0.9620    0.9620       999

